In [3]:
using LinearAlgebra

In [4]:
# Example usage:
A = randn(6, 5)

6×5 Matrix{Float64}:
 -1.95631     0.883953    1.91018   -0.149712   -2.01791
 -0.0727755  -0.76632    -0.470417  -0.953192    0.854718
  1.01747     1.56814     0.271173   0.0313981   0.575753
  1.27821     0.638277    1.76257   -0.893859    0.609986
 -1.01789    -0.0390917   1.23846   -0.781792   -0.319784
  1.02494    -1.43909     0.119438   1.37342    -1.35523

In [5]:
function householder(x)
    """Computes the Householder transformation for input vector x
    
    Returns 
    beta: float, the multiplier for future Householder reflection
    v: vector, the householder reflector vector  
    """
    sigma = dot(x[2:end],x[2:end])
    v = copy(x)

    if sigma == 0
        beta = 0
        return beta, v
    end

    sq = sqrt(x[1]^2 + sigma)
    if x[1] > 0
        v[1] += sq
    else
        v[1] -= sq
    end

    beta = 2.0 / (v[1]^2 + sigma)

    return beta, v
end


function apply_householder_col!(B, H, cidx, m)
    """Apply Householder reflection from the left (B<- H * B)
    
    Returns (inplace) 
    B: input matrix
    H: householder vectors v  
    """
    beta, v = householder(B[cidx:m, cidx])
    B[cidx:m, cidx:end] = B[cidx:m, cidx:end] - beta * v * (v' * B[cidx:m, cidx:end])
    H[cidx:end, cidx] = v 

    return B, H
end

function apply_householder_row!(B, H, ridx, n)
    """Apply Householder reflection from the right (B<- B * H)
    
    Returns (inplace) 
    B: input matrix
    H: householder vectors v  
    """
    beta, v = householder(B[ridx, ridx+1:n])
    B[ridx:end, ridx+1:n] = B[ridx:end, ridx+1:n] - (B[ridx:end, ridx+1:n] * v) * (beta * v')
    H[ridx, ridx+1:end] = v 

    return B, H
end



function bidiagonalize(A)
    """Performs matrix bidiagonalization (for future singular value decomposition computation)
    
    Returns (inplace) 
    B: bidiagonal matrix with same singular values as input matrix (B = Q_r * A * Q_l)
    """
    m, n = size(A)
    B = copy(A)
    H = copy(A)  # v vectors in householder 
    
    for k = 1:min(m,n)
        
        # column
        apply_householder_col!(B, H, k, m)
        
        # row
        if k < n
            apply_householder_row!(B, H, k, n) 
        end
    end
    return B, H
end



bidiagonalize (generic function with 1 method)

In [6]:
B, H = bidiagonalize(A);

In [7]:
show(stdout, "text/plain", B)

6×5 Matrix{Float64}:
  2.93054       1.72259       1.11022e-16  -5.55112e-17  -2.22045e-16
 -1.38778e-17  -2.61878      -1.75524      -2.22045e-16   0.0
  2.22045e-16   1.11022e-16   0.796305     -2.70467       4.44089e-16
  4.44089e-16   4.44089e-16   0.0          -1.35604       1.37063
 -2.22045e-16   2.77556e-17   0.0           1.38778e-17   1.41645
  2.22045e-16   2.22045e-16  -1.11022e-16   0.0           0.0

In [8]:
show(stdout, "text/plain", H)

6×5 Matrix{Float64}:
 -4.88685    -1.96053   -0.788935    0.496533    1.42889
 -0.0727755   3.44039    2.95513     1.0602      0.719111
  1.01747    -0.777886  -1.11591     3.39226    -2.61581
  1.27821    -1.80751   -0.0507357   2.60738     1.37063
 -1.01789    -0.128325  -0.112543   -0.0942052  -2.58331
  1.02494    -1.5147     0.718827    0.51393     0.802995

In [409]:
_, S1, _ = svd(B);

In [410]:
S1

5-element Vector{Float64}:
 4.5343535496535186
 3.1563235799851928
 1.7102680994691821
 0.7464249584949121
 0.4035071298801028

In [411]:
_, S2, _ = svd(A);

In [412]:
S2

5-element Vector{Float64}:
 4.5343535496535186
 3.1563235799851936
 1.710268099469183
 0.7464249584949119
 0.4035071298801026

In [413]:
v1 = H[1:end, 1]

6-element Vector{Float64}:
  4.808031481946835
 -0.40435544156476855
 -0.006673547356992271
  0.40780983153278233
 -1.594701211126823
  1.0070139859339338

In [414]:
v1 = v1/norm(v1)

6-element Vector{Float64}:
  0.9252346001790483
 -0.07781222870756112
 -0.001284225559630178
  0.07847697500402746
 -0.30687667978507743
  0.19378495880252133

In [415]:
A2 = A - 2 * v1 * v1'* A

6×5 Matrix{Float64}:
 -2.80824       0.545324    1.20463    -1.09464   -0.769293
 -1.11022e-16  -0.03607    -0.661895    0.242716  -1.31807
 -8.67362e-19  -0.0244923  -0.0262843  -0.159013   2.33936
  5.55112e-17   0.943753   -0.297041    0.782625   2.16064
 -2.22045e-16  -0.776074    0.156714   -1.59123   -2.07281
  0.0          -0.146977    0.855217   -0.725751  -0.115367

In [498]:
function givens_rotation(v)
    a, b = v[1], v[2]
    if b == 0
        c = 1
        s = 0
    else
        if abs(b) > abs(a)
            tau = -a/b
            s = 1.0/sqrt(1.0+tau*tau)
            c = s*tau
        else
            tau = -b/a
            c = 1.0/sqrt(1.0+tau*tau)
            s = c*tau
        end
    end
    return [c -s;s c]
end

givens_rotation (generic function with 2 methods)

In [499]:
A = [1.0 3 4;4 -1 8;7 8 -1]

3×3 Matrix{Float64}:
 1.0   3.0   4.0
 4.0  -1.0   8.0
 7.0   8.0  -1.0

In [500]:
G = givens_rotation(A[[1,2], 1])

2×2 Matrix{Float64}:
 -0.242536  -0.970143
  0.970143  -0.242536

In [501]:
A[[1,2], :] = G * A[[1,2], :];
A

3×3 Matrix{Float64}:
 -4.12311  0.242536  -8.73128
  0.0      3.15296    1.94029
  7.0      8.0       -1.0

In [502]:
G = givens_rotation(A[[1,3], 1])

2×2 Matrix{Float64}:
 0.507519  -0.86164
 0.86164    0.507519

In [503]:
A[[1,3], :] = G * A[[1,3], :];
A


3×3 Matrix{Float64}:
 -8.12404      -6.77003  -3.56965
  0.0           3.15296   1.94029
 -3.33067e-16   4.26913  -8.03075

In [504]:
G = givens_rotation(A[[2,3], 2])

2×2 Matrix{Float64}:
 -0.594089  -0.8044
  0.8044    -0.594089

In [505]:
A[[2,3], :] = G * A[[2,3], :];
A


3×3 Matrix{Float64}:
 -8.12404      -6.77003      -3.56965
  2.67919e-16  -5.30723       5.30723
  1.97871e-16   2.10348e-16   6.33174

In [506]:
qr(A)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}, Matrix{Float64}}
Q factor: 3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}
R factor:
3×3 Matrix{Float64}:
 8.12404  6.77003   3.56965
 0.0      5.30723  -5.30723
 0.0      0.0       6.33174